In [1]:
import sys
sys.path.insert(0,'../')
from libs.ssl_task import RelativePositioning
from libs.ssl_data import SSLHBNDataModule
# import lightning as L
import torch
%load_ext autoreload
%autoreload 2

In [6]:
import wandb
run = wandb.init()
artifact = run.use_artifact('dt-young112/eeg-ssl/model-ndmxtwb2:best', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [7]:
litDataModule = SSLHBNDataModule(target_label='p_factor')
litDataModule.setup(stage='test')
val_data = litDataModule.test_dataloader()
model = RelativePositioning.RelativePositioningLit.load_from_checkpoint("artifacts/model-ndmxtwb2:v0/model.ckpt")

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.


In [9]:
model.eval()
embeds = []
labels = []
for batch in val_data:
    x, y, _ = batch
    z = model.embed(x.to(device='cuda'))
    embeds.append(z.cpu())
    labels.append(y.cpu())
    
embeds = torch.cat(embeds, dim=0).detach().cpu()
labels = torch.cat(labels, dim=0).detach().cpu()

from sklearn.linear_model import LinearRegression
regr = LinearRegression()
regr.fit(embeds, labels)
preds = regr.predict(embeds)


In [ ]:
regr.score(embeds, labels) # best possible score is 1.0

0.9769622683525085

In [11]:
list(zip(preds, labels))

[(-1.3279724, tensor(-1.2340, dtype=torch.float64)),
 (-1.1798935, tensor(-1.2340, dtype=torch.float64)),
 (-1.130127, tensor(-1.2340, dtype=torch.float64)),
 (-1.6467667, tensor(-1.2340, dtype=torch.float64)),
 (-1.1446152, tensor(-1.2340, dtype=torch.float64)),
 (-1.1305962, tensor(-1.2340, dtype=torch.float64)),
 (-1.1209068, tensor(-1.2340, dtype=torch.float64)),
 (-1.0896797, tensor(-1.2340, dtype=torch.float64)),
 (-1.1802559, tensor(-1.2340, dtype=torch.float64)),
 (-1.1976433, tensor(-1.2340, dtype=torch.float64)),
 (-1.0953751, tensor(-1.2340, dtype=torch.float64)),
 (-1.3006248, tensor(-1.2340, dtype=torch.float64)),
 (-1.2373543, tensor(-1.2340, dtype=torch.float64)),
 (-1.1238823, tensor(-1.2340, dtype=torch.float64)),
 (-1.3816566, tensor(-1.2340, dtype=torch.float64)),
 (-1.3261743, tensor(-1.2340, dtype=torch.float64)),
 (-1.1793246, tensor(-1.2340, dtype=torch.float64)),
 (-0.90618134, tensor(-1.2340, dtype=torch.float64)),
 (-1.210681, tensor(-1.2340, dtype=torch.float